In [ ]:
!git clone http://github.com/Zapy67/ATML-PA-4

Cloning into 'ATML-PA-4'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 153 (delta 71), reused 67 (delta 65), pack-reused 74 (from 1)
Receiving objects: 100% (153/153), 356.53 KiB | 8.91 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [ ]:
!git pull http://github.com/Zapy67/ATML-PA-4

From http://github.com/Zapy67/ATML-PA-4
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


In [ ]:
%cd ATML-PA-4

/kaggle/working/ATML-PA-4


In [ ]:
!ls

ATML-PA-4


#  FedProx

In [ ]:
import torch
import torch.nn as nn

In [ ]:
# !pip install fedlab

In [ ]:
import fed_lib
import importlib
importlib.reload(fed_lib)

<module 'fed_lib' from '/kaggle/working/ATML-PA-4/fed_lib/__init__.py'>

In [ ]:
from fed_lib import fed_methods, fed_model, utils
importlib.reload(fed_methods)
importlib.reload(fed_model)
importlib.reload(utils)

17:48:53 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


<module 'fed_lib.utils' from '/kaggle/working/ATML-PA-4/fed_lib/utils.py'>

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import matplotlib.pyplot as plt
def plot_fed_vs_central_acc(metrics, title="FedProx vs Central Test Accuracy"):
    """
    Args:
        metrics (dict): self.round_metrics dictionary from FedSGD.
    """
    fed_acc = metrics.get('fed_test_acc', [])
    central_acc = metrics.get('central_test_acc', [])

    rounds = list(range(1, len(fed_acc) + 1))

    plt.figure(figsize=(8, 5))
    plt.plot(rounds, fed_acc, marker='o', label='FedProx (Server) Accuracy')
    plt.plot(rounds, central_acc, marker='s', label='Central Model Accuracy')

    plt.xlabel("Round")
    plt.ylabel("Accuracy")
    plt.title(title)
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
def plot_fed_test_acc(metrics, title="Federated Test Accuracy"):
    fed_acc = metrics.get('fed_test_acc', [])
    rounds = list(range(1, len(fed_acc) + 1))

    plt.figure(figsize=(8, 5))
    plt.plot(rounds, fed_acc, marker='o', label='Fed Test Accuracy')

    plt.xlabel("Round")
    plt.ylabel("Accuracy")
    plt.title(title)
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
def plot_client_drift(metrics, title="Client Drift Over Rounds"):
    drift = metrics.get('client_drift', [])
    rounds = list(range(1, len(drift) + 1))

    plt.figure(figsize=(8, 5))
    plt.plot(rounds, drift, marker='s', label='Client Drift')

    plt.xlabel("Round")
    plt.ylabel("Drift")
    plt.title(title)
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
def fed_avg_exp(lr=1e-3, local_runs=5, num_clients=5, partition="dirichlet", alpha=10000,device=device, global_runs=20, verbose=False, sampling_frac=1.0):
        
        fed = fed_model.Federation(num_clients=num_clients, federate_method=None, partition=partition, domains=None, alpha=alpha, device=device, batch_size=64, pin_memory=True, num_workers=2)
        client_loaders = fed.client_dataloaders
        client_weights = [len(client.dataset.indices)/len(fed.centralized_train_loader) for client in client_loaders]

        fed_meth = fed_methods.FedProx(local_epochs=local_runs, client_weights=client_weights, sample_fraction=sampling_frac, mu=0.01)

        fed.set_method(fed_meth)
        fed.train(global_runs, lr=lr, verbose=verbose, train_central=False)

        return fed

In [ ]:
f = fed_avg_exp(local_runs=5)
plot_fed_test_acc(f.federated_method.round_metrics, f"FedAvg (k={3} local rounds) Test Accuracy")
plot_client_drift(f.federated_method.round_metrics, f"Client Drift (k={3} local rounds)")